In [314]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import time

browser = wb.Chrome()

In [310]:
# ссылка на 0) федеральные суды 1) приговоры 2) дата поступления 2018-2019 3) по трём нужным статьям - 37843 шт
# вначале ссылки есть аргумент на стартовую позицию и количество документов на странице (макс 20)

url = "https://bsr.sudrf.ru/bigs/portal.html#%7B%22start%22:20,%22rows%22:20,%22uid%22:%221fe4a009-f990-4374-a975-4feaa469b09c%22,%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%201%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20а%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20б%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20в%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20г%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20д%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20е%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20е1%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20ж%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20з%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20и%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20к%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20л%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20м%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20%20п.%20н%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20107%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20107%20%20Часть%201%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20107%20%20Часть%202%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20111%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20111%20%20Часть%204%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%222018%20год%22,%22not%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D,%22start%22:20%7D"

In [193]:
import re

def get_id_from_link(link):
    return re.search(r'id=.+?&', link)[0][3:-1]

In [196]:
def one_page_to_table(page_html):
    df = pd.read_html(page_html)[5]
    soup = BeautifulSoup(page_html)
    urls = [tag.get('href') for tag in soup.find('ul', {'id': 'resultsList'}).find('tbody').find_all('a')]
    df['URL'] = urls
    df['ID'] = df['URL'].apply(get_id_from_link)
    return df

In [262]:
from math import ceil
n_cases = 21847
n_steps = ceil(n_cases / 20)
n_steps

1093

In [350]:
#all_data = pd.DataFrame()
curr_position = len(all_data)
for i in range(curr_position, n_cases+1, 20):
    browser.get('http://www.sudrf.ru/')
    url = f"https://bsr.sudrf.ru/bigs/portal.html#%7B%22start%22:{i},%22rows%22:20,%22uid%22:%221fe4a009-f990-4374-a975-4feaa469b09c%22,%22groups%22:%5B%22Уголовные%20дела%22%5D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%201%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20а%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20б%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20в%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20г%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20д%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20е%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20е1%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20ж%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20з%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20и%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20к%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20л%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20п.%20м%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20105%20%20Часть%202%20%20п.%20н%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20107%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20107%20%20Часть%201%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20107%20%20Часть%202%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20111%5C%22,%5C%22sQuery%5C%22:null%7D,%7B%5C%22name%5C%22:%5C%22u_case_user_article%5C%22,%5C%22operator%5C%22:%5C%22EX%5C%22,%5C%22query%5C%22:%5C%22Статья%20111%20%20Часть%204%5C%22,%5C%22sQuery%5C%22:null%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22Уголовные%20дела%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D,%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22common%5C%22,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22,%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_user_document_num%5C%22,%5C%22operator%5C%22:%5C%22GE%5C%22,%5C%22query%5C%22:%5C%221%5C%22%7D%5D%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22FACET_QUERY###case_user_document_num###Да%22%7D,%7B%22type%22:%22SQ%22,%22queryId%22:%227f9e8ff8-4bc8-46aa-bcbd-f2b3ed5f159f%22,%22operator%22:%22AND%22%7D%5D%7D,%22simpleSearchFieldsBundle%22:null,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_user_doc_result_date%22,%22query%22:%222018%20год%22,%22not%22:false%7D,%7B%22field%22:%22case_user_document_type%22,%22query%22:%22Приговор%22,%22not%22:false%7D,%7B%22field%22:%22case_user_doc_result%22,%22query%22:%22Вынесен%20ПРИГОВОР%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D,%22start%22:{i}%7D"
    browser.get(url)

    element = WebDriverWait(browser, 25).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#resultViewSelectorOuter > a:nth-child(2)"))
    )
    
    while 'resultHeader' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(5)
    
    browser.find_element_by_css_selector('#resultViewSelectorOuter > a:nth-child(2)').click()
    #resultViewSelectorOuter > a:nth-child(2)


    while 'id="resultsList"' not in browser.page_source:
        time.sleep(0.1)
    time.sleep(1)
    data_per_step = one_page_to_table(browser.page_source)
    all_data = pd.concat([all_data, data_per_step])
    print(f'{len(all_data)}/{n_cases}')
    if len(all_data)%100==0:
        all_data.to_excel('cases_data_no_texts.xlsx')
all_data

20420/21847
20440/21847
20460/21847
20480/21847
20500/21847
20520/21847
20540/21847
20560/21847
20580/21847
20600/21847
20620/21847
20640/21847
20660/21847
20680/21847
20700/21847
20720/21847
20740/21847
20760/21847
20780/21847
20800/21847
20820/21847
20840/21847
20860/21847
20880/21847
20900/21847
20920/21847
20940/21847
20960/21847
20980/21847
21000/21847
21020/21847
21040/21847
21060/21847
21080/21847
21100/21847
21120/21847
21140/21847
21160/21847
21180/21847
21200/21847
21220/21847
21240/21847
21260/21847
21280/21847
21300/21847
21320/21847
21340/21847
21360/21847
21380/21847
21400/21847
21420/21847
21440/21847
21460/21847
21480/21847
21500/21847
21520/21847
21540/21847
21560/21847
21580/21847
21600/21847
21620/21847
21640/21847
21660/21847
21680/21847
21700/21847
21720/21847
21740/21847
21760/21847
21780/21847
21800/21847
21820/21847
21840/21847
21847/21847


,№,Аннотация,Номер дела (материала),Тип документа,Статья УК РФ,Дата поступления,Дата решения,Субъект РФ,Наименование суда,ФИО,Результат,URL,ID
0,1,Статья 105 Часть 2 п.г; Статья 105 Часть 2...П...,2-31/2018,Приговор,Статья 105 Часть 2 п.г; Статья 105 Часть 2 п.г,13.09.2018,22.10.2018,Республика Татарстан,Верховный Суд Республики Татарстан,Парахин А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,ec89b941970471362dc925f201c91098
1,2,Статья 105 Часть 2 п.ж; Статья 105 Часть 2 п.п...,1-63/2018 (1-1539/2017;),Приговор,Статья 105 Часть 2 п.ж; Статья 105 Часть 2 п.п...,01.12.2017,01.06.2018,Республика Татарстан,Набережночелнинский городской суд,Хуббатуллин Н.Б.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,1b61023f0dc291d773ca92fe22f85f7d
2,3,Статья 105 Часть 2...Приговор...Вынесен ПРИГОВОР,2-4/2018,Приговор,Статья 105 Часть 2 п.д,29.01.2018,29.03.2018,Тверская область,Тверской областной суд,Тертерян А.Н.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,d6bc8a8bb14a4db4ce7d50ba8073b9fa
3,4,Статья 105 Часть 2...Приговор...Вынесен ПРИГОВОР,1-8/2018 (1-49/2017;),Приговор,Статья 105 Часть 2 п.в,03.11.2017,30.01.2018,Свердловская область,Свердловский областной суд,Поличинский А.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,3b8b13f92c16de8465ace460479d93ef
4,5,Статья 105 Часть 2...Приговор...Вынесен ПРИГОВОР,2-3/2018,Приговор,Статья 105 Часть 2 п.ж,19.01.2018,06.03.2018,Чувашская Республика - Чувашия,Верховный Суд Чувашской Республики,Сидорова М.А.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,080ea8dbc5586681f6308698c13ab858
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,21843,Приговор,1-155/2018,Приговор,Статья 111 Часть 1; Статья 166 Часть 3; Статья...,14.03.2018,04.04.2018,Алтайский край,Железнодорожный районный суд г. Барнаула,Гасымов Рамин Алимович,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,6b80992c55f1b0b904223b0f9ac07fcd
3,21844,Приговор,1-235/2018,Приговор,Статья 162 Часть 1; Статья 158 Часть 1; Статья...,08.02.2018,10.05.2018,Ростовская область,Новочеркасский городской суд,Шапкин М.С.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,8adb02d27b01f3b395e6603afea54e9f
4,21845,Приговор,1-20/2018,Приговор,Статья 162 Часть 1; Статья 162 Часть 3; Статья...,22.01.2018,11.07.2018,Челябинская область,Агаповский районный суд,Бугров С.С.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,0d1e44041212d301ecefe33974775840
5,21846,Приговор,1-120/2018 (1-949/2017;),Приговор,Статья 162 Часть 1; Статья 158 Часть 1; Статья...,29.12.2017,03.05.2018,Свердловская область,Чкаловский районный суд г. Екатеринбурга,Коковин М.В.,Вынесен ПРИГОВОР,https://bsr.sudrf.ru/bigs/showDocument.html#id...,e798003f7334d640b42ac20d6b8d4af4
